In [1]:
from bs4 import BeautifulSoup
import requests
import os
import threading

In [37]:
#scrape root directory from physionet MIMICIII waveform (only)
ROOTURL = "https://physionet.org/files/mimic3wdb/1.0"
recordsRequest = requests.get(ROOTURL + "/RECORDS-waveforms")

#convert request item in Soup obj and into vector of string names
soup = BeautifulSoup(recordsRequest.text, 'html.parser')
#choose random selection of 100 records
import random
all_record_paths = random.sample(str(soup).splitlines(),100)
# all_record_paths = str(soup).splitlines()[0:100]

In [ ]:
idx = 0
folder_root = "physionet.org/files/mimic3wdb/1.0/"

def scrape_mimic_list(record_paths):
    for record_path in record_paths:
        scrape_mimic(record_path=record_path)

def scrape_mimic(record_path):
    print(record_path)
    record_name = record_path[3:10]

    #Will look in each record's general layout file for presence of an ABP and PLETH waveform
    #    if it exists then will download file
    RecordsInFolderRequest = requests.get(ROOTURL + "/" + record_path + "/RECORDS")
    all_records_in_folder = str(BeautifulSoup(RecordsInFolderRequest.text, 'html.parser')).splitlines()

    for record in all_records_in_folder:
        HeaderRequest = requests.get(ROOTURL + "/" + record_path + record + ".hea")
        soup_layout = BeautifulSoup(HeaderRequest.text, 'html.parser')
        signalData = str(soup_layout).splitlines()[1:]
        signalList = []
        for line in signalData:
            signalList.append(line.split()[-1])
        signal_matches = ["ABP", "PLETH"] 
        
        if not all(a in signalList for a in signal_matches):
            continue
        #use --no-parent otherwise you will download all directory files
        filePath = record_path + record
        download_cmd = "wget -r --no-parent " + ROOTURL + "/" + filePath;
        
        # For use with Windows (with WSL) add wsl add beginning of system command
        if os.name!='posix': 
            download_cmd = "wsl "+ download_cmd

        #check if files have already been downloaded otherwise download them
        if not os.path.isfile(folder_root+filePath+".dat"):
            print("DOWNLOADING -- " + filePath )
            os.system(download_cmd +".dat") #fg; echo DOWNLOADED
        else:
            print("ALREADY DOWNLOADED " + filePath+".dat")
        if not os.path.isfile(folder_root+filePath+".hea"):
            print("DOWNLOADING -- " + filePath )
            os.system(download_cmd +".hea") #fg; echo DOWNLOADED
        else:
            print("ALREADY DOWNLOADED " + filePath+".hea")
        print("DOWNLOAD COMPLETE -- " + filePath)


if __name__ == '__main__':
    numThreads = 8
    threadList = []

    numRecords = len(all_record_paths)//numThreads
    
    for x in range(8):
#         threadRunner = threading.Thread(target=scrape_mimic_list, args=(all_record_paths[x:len(all_record_paths):numThreads],))
        if x < 7:
            threadRunner = threading.Thread(target=scrape_mimic_list, args=(all_record_paths[x*numRecords:(x+1)*numRecords],))
        else:
            threadRunner = threading.Thread(target=scrape_mimic_list, args=(all_record_paths[x*numRecords:],))
        threadRunner.start()
        threadList.append(threadRunner)

    for x in range(8):
        print("joining threads....")
        threadList[x].join()

39/3950773/32/3295493/

34/3442093/
37/3767006/34/3441291/

31/3128332/36/3692433/

35/3534350/joining threads....

36/3600482/
DOWNLOADING -- 39/3950773/3950773_0004
DOWNLOADING -- 39/3950773/3950773_0004
38/3877931/
36/3624102/
DOWNLOAD COMPLETE -- 39/3950773/3950773_0004
DOWNLOADING -- 39/3950773/3950773_0005
32/3206256/
DOWNLOADING -- 39/3950773/3950773_0005
DOWNLOAD COMPLETE -- 39/3950773/3950773_0005
30/3017276/
35/3582515/
36/3673535/
DOWNLOADING -- 30/3017276/3017276_0003
DOWNLOADING -- 30/3017276/3017276_0003
38/3803446/
DOWNLOAD COMPLETE -- 30/3017276/3017276_0003
31/3161587/
36/3653018/
DOWNLOADING -- 31/3161587/3161587_0006
DOWNLOADING -- 31/3161587/3161587_0006
DOWNLOAD COMPLETE -- 31/3161587/3161587_0006
DOWNLOADING -- 31/3161587/3161587_0007
31/3114985/
36/3691095/
DOWNLOADING -- 34/3441291/3441291_0042
DOWNLOADING -- 31/3161587/3161587_0007
DOWNLOADING -- 34/3441291/3441291_0042
DOWNLOAD COMPLETE -- 31/3161587/3161587_0007
DOWNLOAD COMPLETE -- 34/3441291/3441291_0042DOW

DOWNLOADING -- 34/3453522/3453522_0012
DOWNLOADING -- 30/3096468/3096468_0012
DOWNLOADING -- 34/3442093/3442093_0210
DOWNLOAD COMPLETE -- 39/3968719/3968719_0015
DOWNLOADING -- 39/3968719/3968719_0016
30/3094457/
DOWNLOADING -- 37/3739816/3739816_0016
32/3246424/
DOWNLOADING -- 32/3246424/3246424_0002
DOWNLOADING -- 31/3161587/3161587_0028
DOWNLOADING -- 34/3453522/3453522_0012
DOWNLOAD COMPLETE -- 30/3096468/3096468_0012
DOWNLOAD COMPLETE -- 34/3442093/3442093_0210
DOWNLOADING -- 30/3096468/3096468_0014
DOWNLOADING -- 39/3968719/3968719_0016DOWNLOADING -- 34/3442093/3442093_0211

DOWNLOAD COMPLETE -- 37/3739816/3739816_0016
DOWNLOADING -- 32/3246424/3246424_0002
DOWNLOAD COMPLETE -- 31/3161587/3161587_0028
DOWNLOADING -- 37/3739816/3739816_0018
DOWNLOAD COMPLETE -- 34/3453522/3453522_0012
DOWNLOADING -- 31/3161587/3161587_0029
DOWNLOADING -- 34/3453522/3453522_0013
DOWNLOADING -- 30/3096468/3096468_0014
DOWNLOAD COMPLETE -- 39/3968719/3968719_0016
DOWNLOADING -- 39/3968719/3968719_001

DOWNLOAD COMPLETE -- 37/3781159/3781159_0007
DOWNLOAD COMPLETE -- 32/3246424/3246424_0014
DOWNLOADING -- 37/3781159/3781159_0009
DOWNLOAD COMPLETE -- 30/3096468/3096468_0023
DOWNLOADING -- 32/3246424/3246424_0016
DOWNLOADING -- 30/3096468/3096468_0024
DOWNLOADING -- 31/3161587/3161587_0040
DOWNLOAD COMPLETE -- 33/3344507/3344507_0003
DOWNLOADING -- 34/3453522/3453522_0022
DOWNLOADING -- 33/3344507/3344507_0005
DOWNLOAD COMPLETE -- 34/3442093/3442093_0220
DOWNLOADING -- 34/3442093/3442093_0221
DOWNLOADING -- 37/3781159/3781159_0009
DOWNLOADING -- 32/3246424/3246424_0016
DOWNLOADING -- 30/3096468/3096468_0024
DOWNLOAD COMPLETE -- 31/3161587/3161587_0040
DOWNLOADING -- 31/3161587/3161587_0041
DOWNLOAD COMPLETE -- 34/3453522/3453522_0022
DOWNLOADING -- 34/3453522/3453522_0023
DOWNLOADING -- 33/3344507/3344507_0005
DOWNLOADING -- 34/3442093/3442093_0221
DOWNLOAD COMPLETE -- 37/3781159/3781159_0009
DOWNLOADING -- 37/3781159/3781159_0010
DOWNLOAD COMPLETE -- 32/3246424/3246424_0016
DOWNLOAD C

In [23]:
folderRoot = "physionet.org/files/mimic3wdb/1.0/"

dataFileList = []
for rootfolder, folder, filenames in os.walk('./physionet.org/files/mimic3wdb/1.0/'):
    for filename in [file for file in filenames if file.endswith(".dat")]:
        dataFileList.append(os.path.join(filename[0:2],filename[0:7], filename[0:12]))

fileThreshold = 17000
for record in dataFileList:
    filePath = folderRoot+record
    fileSize =  os.path.getsize(filePath+'.dat')
    if fileSize > fileThreshold:
        # Convert to TXT
        #rdsamp: to text is -p > newName.txt
        # -s : signal list is ABP, PLETH in that order => output cols will be TIME-ABP-PLETH
        # -S : search for first valid time for ABP
        rdsamp_cmd = "rdsamp -r " + filePath +" -p > " + filePath + ".txt -s ABP PLETH -S ABP"
        if os.name!='posix': 
            rdsamp_cmd = "wsl " + rdsamp_cmd
        print('CONVERTING TO TXT -- ' + filePath)
        print(rdsamp_cmd)
        os.system(rdsamp_cmd)
        print('CONVERTING TO TXT COMPLETE -- ' + filePath)
        # move converted files to new folder
    
        move_cmd = "mv " + filePath +".txt physionet.org/textdata/"
        if os.name != 'posix':
            move_cmd = "wsl "+ move_cmd
        print(move_cmd)
        os.system( move_cmd )
        
        # Convert to MAT
        # wfdb2mat_cmd = "wfdb2mat -r " + filePath
        # print('CONVERTING TO MAT -- ' + filePath)
        # if os.name!='posix':
        #     wfdb2mat_cmd = "wsl "+wfdb2mat_cmd
        # os.system(wfdb2mat_cmd)
        # folder = record.split('/')[0] + '/' + record.split('/')[1] + '/'
        # fileName = record.split('/')[2]
        # mv_cmd = "mv " + fileName + "m.mat " + fileName + "m.hea " + folderRoot + folder 
        # if os.name!='posix': 
        #     mv_cmd = "wsl "+ mv_cmd
        # os.system(mv_cmd)
        # print('CONVERTING TO MAT COMPLETE -- ' + filePath)
        

CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0012
rdsamp -r physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0012 -p > physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0012.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0012
mv physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0012.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0007
rdsamp -r physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0007 -p > physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0007.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0007
mv physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0007.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0005
rdsamp -r physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0005 -p > physionet.org/files/mimic3

CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/34/3442093/3442093_0211
mv physionet.org/files/mimic3wdb/1.0/34/3442093/3442093_0211.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/34/3475182/3475182_0001
rdsamp -r physionet.org/files/mimic3wdb/1.0/34/3475182/3475182_0001 -p > physionet.org/files/mimic3wdb/1.0/34/3475182/3475182_0001.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/34/3475182/3475182_0001
mv physionet.org/files/mimic3wdb/1.0/34/3475182/3475182_0001.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/34/3475182/3475182_0008
rdsamp -r physionet.org/files/mimic3wdb/1.0/34/3475182/3475182_0008 -p > physionet.org/files/mimic3wdb/1.0/34/3475182/3475182_0008.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/34/3475182/3475182_0008
mv physionet.org/files/mimic3wdb/1.0/34/3475182/3475182_0008.txt physionet.org/textdata/
C

CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0007
mv physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0007.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0006
rdsamp -r physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0006 -p > physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0006.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0006
mv physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0006.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0010
rdsamp -r physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0010 -p > physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0010.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0010
mv physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0010.txt physionet.org/textdata/
C

CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000989/3000989_0002
mv physionet.org/files/mimic3wdb/1.0/30/3000989/3000989_0002.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3000989/3000989_0004
rdsamp -r physionet.org/files/mimic3wdb/1.0/30/3000989/3000989_0004 -p > physionet.org/files/mimic3wdb/1.0/30/3000989/3000989_0004.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000989/3000989_0004
mv physionet.org/files/mimic3wdb/1.0/30/3000989/3000989_0004.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3001262/3001262_0009
rdsamp -r physionet.org/files/mimic3wdb/1.0/30/3001262/3001262_0009 -p > physionet.org/files/mimic3wdb/1.0/30/3001262/3001262_0009.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3001262/3001262_0009
mv physionet.org/files/mimic3wdb/1.0/30/3001262/3001262_0009.txt physionet.org/textdata/
C

CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3002324/3002324_0013
mv physionet.org/files/mimic3wdb/1.0/30/3002324/3002324_0013.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3002324/3002324_0014
rdsamp -r physionet.org/files/mimic3wdb/1.0/30/3002324/3002324_0014 -p > physionet.org/files/mimic3wdb/1.0/30/3002324/3002324_0014.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3002324/3002324_0014
mv physionet.org/files/mimic3wdb/1.0/30/3002324/3002324_0014.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0010
rdsamp -r physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0010 -p > physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0010.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0010
mv physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0010.txt physionet.org/textdata/
C

CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0012
mv physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0012.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0002
rdsamp -r physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0002 -p > physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0002.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0002
mv physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0002.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0016
rdsamp -r physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0016 -p > physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0016.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0016
mv physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0016.txt physionet.org/textdata/
C

CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0015
mv physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0015.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0029
rdsamp -r physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0029 -p > physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0029.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0029
mv physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0029.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0014
rdsamp -r physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0014 -p > physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0014.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0014
mv physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0014.txt physionet.org/textdata/
C

CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000860/3000860_0007
mv physionet.org/files/mimic3wdb/1.0/30/3000860/3000860_0007.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3000860/3000860_0012
rdsamp -r physionet.org/files/mimic3wdb/1.0/30/3000860/3000860_0012 -p > physionet.org/files/mimic3wdb/1.0/30/3000860/3000860_0012.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000860/3000860_0012
mv physionet.org/files/mimic3wdb/1.0/30/3000860/3000860_0012.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3002541/3002541_0002
rdsamp -r physionet.org/files/mimic3wdb/1.0/30/3002541/3002541_0002 -p > physionet.org/files/mimic3wdb/1.0/30/3002541/3002541_0002.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3002541/3002541_0002
mv physionet.org/files/mimic3wdb/1.0/30/3002541/3002541_0002.txt physionet.org/textdata/
C

CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0005
mv physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0005.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0010
rdsamp -r physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0010 -p > physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0010.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0010
mv physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0010.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0012
rdsamp -r physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0012 -p > physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0012.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0012
mv physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0012.txt physionet.org/textdata/
C

CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0020
mv physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0020.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0018
rdsamp -r physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0018 -p > physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0018.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0018
mv physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0018.txt physionet.org/textdata/
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0019
rdsamp -r physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0019 -p > physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0019.txt -s ABP PLETH -S ABP
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0019
mv physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0019.txt physionet.org/textdata/
C

In [24]:
# Generate wabp files and extract to text:
txtFileList = os.listdir('physionet.org/textdata/')
fileNames = [f[:-4] for f in txtFileList]

ROOTFOLDER = "physionet.org/files/mimic3wdb/1.0"
EXTRACTFOLDER = "physionet.org/abp_ann"
if not os.path.isdir(EXTRACTFOLDER):
    os.mkdir(EXTRACTFOLDER)


for f in fileNames:
    wabp_cmd = "wabp -r " + ROOTFOLDER + "/" + f[:2] + "/" + f[:7] + "/" +  f
    if os.name != 'posix':
        wabp_cmd = "wsl "+ wabp_cmd
    print(wabp_cmd)
    os.system(wabp_cmd)
    wabp_cmd = "rdann -r " + ROOTFOLDER + "/" + f[:2] + "/" + f[:7] + "/" +  f + " -a wabp > physionet.org/abp_ann/" + f+"_abp.txt"
    print(wabp_cmd)
    if os.name != 'posix':
        wabp_cmd = "wsl "+ wabp_cmd
    os.system(wabp_cmd)  

wabp -r physionet.org/files/mimic3wdb/1.0/34/3453522/3453522_0007
rdann -r physionet.org/files/mimic3wdb/1.0/34/3453522/3453522_0007 -a wabp > physionet.org/abp_ann/3453522_0007_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0012
rdann -r physionet.org/files/mimic3wdb/1.0/32/3246424/3246424_0012 -a wabp > physionet.org/abp_ann/3246424_0012_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0013
rdann -r physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0013 -a wabp > physionet.org/abp_ann/3000063_0013_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0007
rdann -r physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0007 -a wabp > physionet.org/abp_ann/3000063_0007_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3000781/3000781_0004
rdann -r physionet.org/files/mimic3wdb/1.0/30/3000781/3000781_0004 -a wabp > physionet.org/abp_ann/3000781_0004_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3035686/3035686_0014
rdan

rdann -r physionet.org/files/mimic3wdb/1.0/30/3035686/3035686_0010 -a wabp > physionet.org/abp_ann/3035686_0010_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3000484/3000484_0004
rdann -r physionet.org/files/mimic3wdb/1.0/30/3000484/3000484_0004 -a wabp > physionet.org/abp_ann/3000484_0004_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3017276/3017276_0003
rdann -r physionet.org/files/mimic3wdb/1.0/30/3017276/3017276_0003 -a wabp > physionet.org/abp_ann/3017276_0003_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/34/3453522/3453522_0003
rdann -r physionet.org/files/mimic3wdb/1.0/34/3453522/3453522_0003 -a wabp > physionet.org/abp_ann/3453522_0003_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0002
rdann -r physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0002 -a wabp > physionet.org/abp_ann/3000393_0002_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3096468/3096468_0017
rdann -r physionet.org/files/mimic3wdb/1.0/30/3096468/3096468_0017 -a 

rdann -r physionet.org/files/mimic3wdb/1.0/30/3096468/3096468_0012 -a wabp > physionet.org/abp_ann/3096468_0012_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0016
rdann -r physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0016 -a wabp > physionet.org/abp_ann/3161587_0016_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0012
rdann -r physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0012 -a wabp > physionet.org/abp_ann/3739816_0012_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0006
rdann -r physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0006 -a wabp > physionet.org/abp_ann/3739816_0006_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0013
rdann -r physionet.org/files/mimic3wdb/1.0/30/3000714/3000714_0013 -a wabp > physionet.org/abp_ann/3000714_0013_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/37/3781159/3781159_0004
rdann -r physionet.org/files/mimic3wdb/1.0/37/3781159/3781159_0004 -a 

wabp -r physionet.org/files/mimic3wdb/1.0/36/3600271/3600271_0023
rdann -r physionet.org/files/mimic3wdb/1.0/36/3600271/3600271_0023 -a wabp > physionet.org/abp_ann/3600271_0023_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0031
rdann -r physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0031 -a wabp > physionet.org/abp_ann/3000393_0031_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0025
rdann -r physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0025 -a wabp > physionet.org/abp_ann/3000393_0025_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0019
rdann -r physionet.org/files/mimic3wdb/1.0/30/3000393/3000393_0019 -a wabp > physionet.org/abp_ann/3000393_0019_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0034
rdann -r physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0034 -a wabp > physionet.org/abp_ann/3161587_0034_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/37/3739816/3739816_0018
rdan

rdann -r physionet.org/files/mimic3wdb/1.0/34/3453522/3453522_0021 -a wabp > physionet.org/abp_ann/3453522_0021_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3002324/3002324_0013
rdann -r physionet.org/files/mimic3wdb/1.0/30/3002324/3002324_0013 -a wabp > physionet.org/abp_ann/3002324_0013_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0040
rdann -r physionet.org/files/mimic3wdb/1.0/31/3161587/3161587_0040 -a wabp > physionet.org/abp_ann/3161587_0040_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0023
rdann -r physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0023 -a wabp > physionet.org/abp_ann/3000063_0023_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0017
rdann -r physionet.org/files/mimic3wdb/1.0/39/3968719/3968719_0017 -a wabp > physionet.org/abp_ann/3968719_0017_abp.txt
wabp -r physionet.org/files/mimic3wdb/1.0/30/3001262/3001262_0007
rdann -r physionet.org/files/mimic3wdb/1.0/30/3001262/3001262_0007 -a 